### 점진적인 학습
#### 확률적 경사 하강법
- 확률적이란 말은 '무작위하게' 혹은 '랜덤하게' 라는 뜻
- 훈련 세트에서 랜덤하게 하나의 샘플을 고르는 것
- 에포크: 훈련 세트를 한 번 모두 사용하는 과정
- 미니배치 경사 하강법: 여러 개의 샘플을 사용해 경사 하강법을 수행하는 방식
- 배치 경사 하강법: 전체 샘플을 사용하는 방법(컴퓨터 자원을 많이 사용함)

#### 손실함수
- 머신러닝 알고리즘이 얼마나 엉터리인지를 측정하는 기준

### 로지스틱 손실 함수
- 로지스틱 손실 함수: 이진 분류에서 사용하는 손실 함수
- 크로스엔트로피 손실 함수: 다중 분류에서 사용하는 손실 함수
- 평균 제곱 오차: 타깃에서 예측을 뺀 값을 제곱한 다음 모든 샘플에 평균한 값. 작을 수록 좋은 모델

### SGDClassifier

In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')

In [3]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [18]:
fish_input = fish.iloc[:,1:6].to_numpy()
fish_target = fish['Species'].to_numpy()

In [19]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [22]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [23]:
# 확률적 경사 하강법 클래스 = SGDClassifier
from sklearn.linear_model import SGDClassifier

In [24]:
sc = SGDClassifier(loss='log', max_iter=10, random_state=42) # loss=log, 로지스틱 손실 함수, max_iter 수행할 에포크 횟수
sc.fit(train_scaled, train_target)
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.773109243697479
0.775


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [25]:
sc.partial_fit(train_scaled, train_target) # 1 에포크씩 이어서 훈련 가능함.
print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.8151260504201681
0.85
